In [2]:
import pandas as pd

In [3]:
%matplotlib inline

## Read data and separate Switzerland, Cantons, Districts, Communes

### Load the Excel with all the data in it

In [4]:
df = pd.read_excel('ReadinData/px-x-0102010000_103 - Zivilstand.xlsx', sheetname='relrel')

In [5]:
df.head(2)

Nummer   Einheit     Ledig  Verheiratet  Geschieden  Wervitwet
0   8100   Schweiz  0.000000     0.000000    0.000000   0.000000
1     AG  - Aargau -3.648455     5.571658   -5.772407  -6.593801

In [6]:
#Make sure we don't have any NaN in there
df = df[df['Einheit'].notnull()]

### Load some helper files

In [7]:
#Load the numbers and abbreviations file for the cantons
df_kant_ids = pd.read_csv('HelperData/Kantone-ABK-ID.csv')
df_kant_ids.head(2)

Kanton_ABK  Kanton_ID Kanton_Name  Kanton_Name2
0         ZH          1      Zürich        Zürich
1         BE          2        Bern  Bern / Berne

In [8]:
#Load the canton/district infos for the communes
df_gem_ids = pd.read_excel('HelperData/Gemeindestand-2015.xls')
df_gem_ids.head(2)

Hist.-Nummer Kanton  Bezirksnummer Bezirksname  BFS Gde-nummer  \
0         13256     ZH            101   Affoltern               1   
1         11742     ZH            101   Affoltern               2   

         Gemeindename Datum der Aufnahme  
0     Aeugst am Albis         1976-11-15  
1  Affoltern am Albis         1960-01-01

In [9]:
df_gem_ids.pop('Hist.-Nummer')
df_gem_ids.pop('Gemeindename')
df_gem_ids.pop('Datum der Aufnahme')
df_gem_ids.rename(columns={'Bezirksnummer': 'Bezirk_ID', 'Bezirksname': 'Bezirk_Name', 'BFS Gde-nummer': 'Gemeinde_ID', 'Kanton': 'Kanton_ABK'}, inplace=True)
df_gem_ids.head(2)

Kanton_ABK  Bezirk_ID Bezirk_Name  Gemeinde_ID
0         ZH        101   Affoltern            1
1         ZH        101   Affoltern            2

In [10]:
#Construct a set of district ids
df_bez_ids = df_gem_ids.drop_duplicates(subset='Bezirk_ID')
df_bez_ids.pop('Gemeinde_ID')
df_bez_ids.head(2)

Kanton_ABK  Bezirk_ID  Bezirk_Name
0          ZH        101    Affoltern
14         ZH        102  Andelfingen

## Get a dataframe for each geopraphic level

### Separate Switzerland

In [11]:
#Switzerland
df_ch = df[df['Einheit'] == 'Schweiz']
df_ch.pop('Einheit')
df_ch.pop('Nummer')
df_ch.head(2)

Ledig  Verheiratet  Geschieden  Wervitwet
0    0.0          0.0         0.0        0.0

In [12]:
df_ch.shape

(1, 4)

### Separate the cantons

In [13]:
#Cantons - there should be 26 of them
df_kant = df[df['Einheit'].str.contains('^- ')]
df_kant['Kanton_Name'] = df_kant['Einheit'].str.extract('^- (.*)', expand=True)
df_kant.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Nummer                   Einheit     Ledig  Verheiratet  Geschieden  \
1     AG                  - Aargau -3.648455     5.571658   -5.772407   
2     AR  - Appenzell Ausserrhoden -3.588657     2.756388    0.109405   

   Wervitwet             Kanton_Name  
1  -6.593801                  Aargau  
2   7.731230  Appenzell Ausserrhoden

In [14]:
#Add the ID and Abbreviations
df_kant = df_kant.merge(df_kant_ids, how='inner', left_on='Kanton_Name', right_on='Kanton_Name2')
df_kant.pop('Kanton_Name2')
df_kant.pop('Einheit')
df_kant.pop('Kanton_Name_y')
df_kant.pop('Nummer')
df_kant.rename(columns={'Kanton_Name_x': 'Kanton_Name'}, inplace=True)
df_kant.head(2)

Ledig  Verheiratet  Geschieden  Wervitwet             Kanton_Name  \
0 -3.648455     5.571658   -5.772407  -6.593801                  Aargau   
1 -3.588657     2.756388    0.109405   7.731230  Appenzell Ausserrhoden   

  Kanton_ABK  Kanton_ID  
0         AG         19  
1         AR         15

In [15]:
df_kant.shape

(26, 7)

### Separate the districts

In [16]:
#Districts - there should be 148 of them
df_bez = df[df['Einheit'].str.contains('^>> ')]
df_bez['Bezirk_Name'] = df_bez['Einheit'].str.extract('^>> (.*)', expand=True)
df_bez['Bezirk_ID'] = df_bez['Nummer'].str.extract('00(.*)', expand=True).astype(int)
df_bez.pop('Nummer')
df_bez.pop('Einheit')
df_bez.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Ledig  Verheiratet  Geschieden  Wervitwet  \
2351 -5.302440     1.602578    3.521495  27.493064   
2352 -0.320606     1.171805   -6.095757   2.825711   

                                    Bezirk_Name  Bezirk_ID  
2351  Arrondissement administratif Jura bernois        241  
2352                               Bezirk Aarau       1901

In [17]:
df_bez.shape

(148, 6)

### Separate the communes

In [18]:
#Gemeinden - there should be 2324 of them
df_gem = df[df['Einheit'].str.contains('^\.\.\.\.\.\.')]
df_gem['Gemeinde_ID'] = df_gem['Einheit'].str.extract('^\.\.\.\.\.\.(\d\d\d\d)', expand=True).astype(int)
df_gem['Gemeinde_Name'] = df_gem['Einheit'].str.extract('^\.\.\.\.\.\.\d\d\d\d (.*)', expand=True)
df_gem.pop('Nummer')
df_gem.pop('Einheit')
df_gem.head(2)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Ledig  Verheiratet  Geschieden  Wervitwet  Gemeinde_ID  \
27 -1.902679      6.51872   -9.880150 -23.653341            1   
28 -2.593191      2.40175    4.145042  -4.861439            2   

         Gemeinde_Name  
27     Aeugst am Albis  
28  Affoltern am Albis

In [19]:
df_gem.shape

(2324, 6)

## Export the data

In [20]:
#Switzerland
df_ch.to_csv('StructuredData/CH-Zivilstand.csv', index=False)
df_temp = pd.read_csv('StructuredData/CH-Zivilstand.csv')
df_temp.head(2)

Ledig  Verheiratet  Geschieden  Wervitwet
0    0.0          0.0         0.0        0.0

In [21]:
#Cantons
df_kant.to_csv('StructuredData/kant-Zivilstand.csv', index=False)
df_temp = pd.read_csv('StructuredData/kant-Zivilstand.csv')
df_temp.head(2)

Ledig  Verheiratet  Geschieden  Wervitwet             Kanton_Name  \
0 -3.648455     5.571658   -5.772407  -6.593801                  Aargau   
1 -3.588657     2.756388    0.109405   7.731230  Appenzell Ausserrhoden   

  Kanton_ABK  Kanton_ID  
0         AG         19  
1         AR         15

In [22]:
#Districts
df_bez.to_csv('StructuredData/bez-Zivilstand.csv', index=False)
df_temp = pd.read_csv('StructuredData/bez-Zivilstand.csv')
df_temp.head(2)

Ledig  Verheiratet  Geschieden  Wervitwet  \
0 -5.302440     1.602578    3.521495  27.493064   
1 -0.320606     1.171805   -6.095757   2.825711   

                                 Bezirk_Name  Bezirk_ID  
0  Arrondissement administratif Jura bernois        241  
1                               Bezirk Aarau       1901

In [23]:
#Communes
df_gem.to_csv('StructuredData/gem-Zivilstand.csv', index=False)
df_temp = pd.read_csv('StructuredData/gem-Zivilstand.csv')
df_temp.head(2)

Ledig  Verheiratet  Geschieden  Wervitwet  Gemeinde_ID  \
0 -1.902679      6.51872   -9.880150 -23.653341            1   
1 -2.593191      2.40175    4.145042  -4.861439            2   

        Gemeinde_Name  
0     Aeugst am Albis  
1  Affoltern am Albis